In [1]:
!pip install angr


In [7]:
import angr
import claripy

# Load an arbitrary file as a "blob" (raw bytes) with Angr
binary_path = '/Users/smander/projects/PHD/binaryAngr/dummy_text.bin'  # This could be any file, even if it's not a true binary
project = angr.Project(binary_path, main_opts={'backend': 'blob', 'arch': 'x86', 'base_addr': 0x400000})

# Create symbolic input and constraints as before
byte1 = claripy.BVS('byte1', 8)
byte2 = claripy.BVS('byte2', 8)
byte3 = claripy.BVS('byte3', 8)
byte4 = claripy.BVS('byte4', 8)

state = project.factory.blank_state()
state.solver.add(byte1 + byte2 + byte3 + byte4 == 240)
for byte in [byte1, byte2, byte3, byte4]:
    state.solver.add(byte >= 0x20)
    state.solver.add(byte <= 0x7e)

# Run symbolic execution
simgr = project.factory.simgr(state)
simgr.explore(find=lambda s: True)

if simgr.found:
    solution = [simgr.found[0].solver.eval(byte, cast_to=bytes) for byte in [byte1, byte2, byte3, byte4]]
    print("Solution found:", solution)
else:
    print("No solution satisfies the constraints.")


WARNING  | 2024-11-10 14:23:42,720 | cle.backends.blob | No entry_point was specified for blob dummy_text.bin, assuming 0


Solution found: [b'.', b'Q', b'1', b'@']
